In [138]:
#https://pythonawesome.com/overview-of-the-peaks-dectection-algorithms-available-in-python/
# https://github.com/MonsieurV/py-findpeaks/blob/master/README.md
import numpy as np
import scipy.signal 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from libs 

In [154]:
from morning.pipeline.stream.cybos.stock.db.min_excel_tick import MinExcelTick
from morning.pipeline.converter.cybos.stock.day_tick import StockDayTickConverter
import pandas as pd

class Receiver:
    def __init__(self):
        self.data = []
    
    def received(self, datas):
        for d in datas:
            self.data.append(d)

sdtc = StockDayTickConverter()
met = MinExcelTick('sample_data/20191129_A028300.xlsx')
met.set_output(sdtc)
rcv = Receiver()
sdtc.set_output(rcv)

while met.received(None) > 0:
    pass

data = rcv.data
df = pd.DataFrame(data)
df['avg'] = df['close_price'].rolling(20).mean()
not_non = df['avg'].notna().idxmax()
if not_non > 0:
    print(df['avg'].iloc[not_non])
    df['avg'] = df['avg'].fillna(df['avg'].iloc[not_non])

118410.0


In [155]:
def get_reversed(s):
    distance_from_mean = s.mean() - s
    return distance_from_mean + s.mean()

In [156]:
# find_peaks_cwt
def get_indexes(methods):
    price = df['avg']
    reversed = get_reversed(price)
    if methods == 'peakutil':
        import peakutils.peak
        indexes = peakutils.peak.indexes(np.array(price), thres=7.0/price.max(), min_dist=10)
        r_indexes = peakutils.peak.indexes(np.array(reversed), thres=7.0/price.max(), min_dist=10)
        return indexes, r_indexes
    elif methods == 'find_peaks_cwt':
        indexes = scipy.signal.find_peaks_cwt(np.array(price), np.arange(1, 4), max_distances=np.arange(1, 4)*2)
        r_indexes = scipy.signal.find_peaks_cwt(np.array(reversed), np.arange(1, 4), max_distances=np.arange(1, 4)*2)
        return indexes, r_indexes
    elif methods == 'find_peaks':
        indexes, _ = scipy.signal.find_peaks(np.array(price), height=7, distance=2.1)
        r_indexes, _ = scipy.signal.find_peaks(np.array(reversed), height=7, distance=2.1)
        return indexes, r_indexes

In [157]:
def show_graph(max_index, min_index):
    min_index_dates = df['date'].iloc[min_index]
    min_index_prices = df['avg'].iloc[min_index] + (df['avg'].mean() * 0.01)
    max_index_dates = df['date'].iloc[max_index]
    max_index_prices = df['avg'].iloc[max_index] + (df['avg'].mean() * 0.01)

    fig = make_subplots(rows=2, cols=1)

    fig.add_trace(go.Scatter(x=df['date'], y=df['close_price'], mode='lines', line=dict(color='black', width=1), opacity=0.9))
    fig.add_trace(go.Scatter(x=df['date'], y=df['avg'], mode='lines', opacity=1.0))
    fig.add_trace(go.Scatter(mode='markers', x=min_index_dates, y=min_index_prices, marker=dict(color='blue', size=7), opacity=0.7))
    fig.add_trace(go.Scatter(mode='markers', x=max_index_dates, y=max_index_prices, marker=dict(color='yellow', size=7), opacity=0.7))
    fig.add_trace(go.Scatter(x=df['date'], y=df['volume'], name='culmulate buy', line=dict(color='red')), row=2, col=1)
    return fig

#fig.update_layout(yaxis_tickformat='d', shapes=shapes)

In [158]:

show_graph(*get_indexes('find_peaks'))
